In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector


In [2]:
from dotenv import load_dotenv

load_dotenv()
import os

def create_connection_string():
    """db接続文字列を生成"""
    DB_HOST = os.getenv("PGHOST")
    DB_PORT = int(os.getenv("PGPORT"))
    DB_NAME = os.getenv("PGDATABASE")
    DB_USER = os.getenv("PGUSER")
    DB_PASS = os.getenv("PGPASSWORD")

    return f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


In [3]:
loader = PyPDFLoader("./BCG-x-SIA-Strengthening-the-Global-Semiconductor-Value-Chain-April-2021_1.pdf")
pages = loader.load_and_split()
embeddings = OpenAIEmbeddings()

vector_store = PGVector.from_documents(
    documents=pages,
    embedding=embeddings,
    collection_name="pages",
    connection_string=create_connection_string()
)


In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

embeddings = OpenAIEmbeddings()

vector_store = PGVector(
    connection_string=create_connection_string(),
    embedding_function=embeddings,
    collection_name="pages",
)
qna = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(temperature=0.0, model="gpt-4"),
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

res = qna(
    {
        "question": "半導体サプライチェーンのチョークポイントは" + "\n\nanswer in Japanese.",
    },
    return_only_outputs=True,
)
content = res["answer"]
if "sources" in res and res["sources"]:
    content += "\n" + "sources:" + "\n" + "\n".join(res["sources"].split(","))



In [10]:
res["answer"]

'半導体サプライチェーンのチョークポイントは、その高度に複雑な製品設計と製造プロセスにあります。現代のチップの開発は、ハードウェアとソフトウェアの両方に深い技術的専門知識を必要とし、高度な設計ツールと専門企業によって提供される知的財産（IP）に依存しています。その後の製造プロセスでは、原材料のウェハー、一般的な化学物質、特殊化学物質、バルクガスなど、約300種類の異なる入力が必要となります。これらの入力は、リソグラフィーや計量ツールなど、50以上のクラスの高精度な工学機器によって処理されます。このような高度に専門化したサプライヤーは、しばしば異なる国に拠点を置いています。その後、チップは世界中を行き来するグローバルな旅をします。このような複雑なグローバルな半導体サプライチェーンの理解は、業界が継続的なパフォーマンスとコスト改善を提供する能力に影響を与える可能性のあるリスクを特定し、それらに対処する方法を議論するために重要です。\n'